## Final visualization code for Pale Blue Dot challenge

### data from source: https://disc.gsfc.nasa.gov/datasets/M2TMNXOCN_5.12.4/summary 

All images included in final report will be found here, and are saved to /reports/figures

## Plotting final sea ice plot for report

In [11]:
import xarray as xr
import plotly.graph_objects as go
import os
import pandas as pd

# Base directory where the NetCDF files are stored
data_dir = '/Users/casey/Desktop/blue-dot/data/raw'

# Months (1 through 12) and their corresponding names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = list(range(1, 13))

# Decade ranges
decades = [(1981, 1990), (1991, 2000), (2001, 2010), (2011, 2022)]

# Custom line and fill colors for each decade
decade_colors = ['#b1ffff', '#00ebeb', '#00c4c4', '#009d9d']

# Latitude and longitude bounds for Northern Alaska
lat_bounds = [64, 71]
lon_bounds = [-168, -141]

# Initialize DataFrame for sea ice extent
df_sea_ice = pd.DataFrame()

# Process each file in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith('.nc4'):
        parts = file_name.split('.')
        date_part = parts[-2]
        year = int(date_part[:4])
        month = int(date_part[4:6])

        file_path = os.path.join(data_dir, file_name)
        ds = xr.open_dataset(file_path)

        sea_ice = ds['FRSEAICE'].sel(lat=slice(*lat_bounds), lon=slice(*lon_bounds))
        df_sea_ice.at[pd.to_datetime(f'{year}-{month}-01'), 'Sea Ice Extent'] = sea_ice.mean().values
        ds.close()

# Create a figure for plotting
fig = go.Figure()

# Plot for each decade
for (start_year, end_year), color in zip(decades, decade_colors):
    decade_data = df_sea_ice[(df_sea_ice.index.year >= start_year) & (df_sea_ice.index.year <= end_year)]
    lower_bound = decade_data.groupby(decade_data.index.month).quantile(0.25) # 25th percentile
    upper_bound = decade_data.groupby(decade_data.index.month).quantile(0.75) # 75th percentile

    fig.add_trace(go.Scatter(x=lower_bound.index, y=lower_bound['Sea Ice Extent'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 25th Pctl', showlegend=False))
    fig.add_trace(go.Scatter(x=upper_bound.index, y=upper_bound['Sea Ice Extent'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 75th Pctl', fill='tonexty', fillcolor=color, showlegend=False))

# Plot the year 2023 separately
data_2023 = df_sea_ice[df_sea_ice.index.year == 2023].sort_index()
if not data_2023.empty:
    fig.add_trace(go.Scatter(x=data_2023.index.month, y=data_2023['Sea Ice Extent'], mode='lines', line=dict(color='#007676', width=2), name='Year 2023'))

# Update layout with custom legend
fig.update_layout(
    title='',
    xaxis=dict(title='Month', tickmode='array', tickvals=months, ticktext=month_names, showgrid=False, title_font=dict(size=16), tickfont=dict(size=16)),
    yaxis=dict(title='Average Sea Ice Fraction (Extent)', showgrid=False, title_font=dict(size=16), tickfont=dict(size=16)),
    plot_bgcolor='rgba(255,255,255,1)',  # White background
    paper_bgcolor='rgba(255,255,255,1)',  
    legend=dict(
        title='Legend',
        orientation='v',
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top', 
        title_font=dict(size=16)
    )
)

# Manually add custom legend entries
for (start_year, end_year), color in zip(decades, decade_colors):
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'{start_year}-{end_year} 25th Percentile', line=dict(color=color, width=10)))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'{start_year}-{end_year} 75th Percentile', line=dict(color=color, width=10)))

fig.show()

fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/sea_ice_frac_plot.svg', width = 1250, height = 700)

## Export just shape of the data for visual

In [12]:
fig.update_layout(showlegend=False, xaxis_visible=False, yaxis_visible=False, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', title='')
fig.show()
fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/sea_ice_frac_shape.svg', width = 1250, height = 700)

## Plotting final water temperature plot for report

In [13]:
import xarray as xr
import plotly.graph_objects as go
import os
import pandas as pd

# Base directory where the NetCDF files are stored
data_dir = '/Users/casey/Desktop/blue-dot/data/raw'

# Months (1 through 12) and their corresponding names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = list(range(1, 13))

# Decade ranges, starting from 1981
decades = [(1981, 1990), (1991, 2000), (2001, 2010), (2011, 2022)]

# Latitude and longitude bounds for Northern Alaska
lat_bounds = [64, 71]
lon_bounds = [-168, -141]

# Initialize DataFrame for water temperature
df_water_temp = pd.DataFrame()

# Process each file in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith('.nc4'):
        # Extract year and month from file name
        parts = file_name.split('.')
        date_part = parts[-2]  # Format 'YYYYMM'
        year = int(date_part[:4])
        month = int(date_part[4:6])

        file_path = os.path.join(data_dir, file_name)
        ds = xr.open_dataset(file_path)

        # Extract 'Var_TSKINWTR' for Northern Alaska region
        water_temp = ds['Var_TSKINWTR'].sel(lat=slice(*lat_bounds), lon=slice(*lon_bounds))

        # Store temperature data in DataFrame
        df_water_temp.at[pd.to_datetime(f'{year}-{month}-01'), 'Water Temperature'] = water_temp.mean().values
        ds.close()

# Create a figure for plotting
fig = go.Figure()

# Custom line and fill colors for each decade
decade_colors = {
    (1981, 1990): '#ff9dff', #pink
    (1991, 2000): '#ffd0b1', #light orange
    (2001, 2010): '#ffc4ff', #medium pink
    (2011, 2022): '#ffb889' #orange
}


# Plot for each decade
for (start_year, end_year), color in decade_colors.items():
    # Select the data for the decade
    decade_data = df_water_temp[(df_water_temp.index.year >= start_year) & (df_water_temp.index.year <= end_year)]
    
    # Calculate the 25th and 75th percentiles for each month
    lower_bound = decade_data.groupby(decade_data.index.month).quantile(0.25)
    upper_bound = decade_data.groupby(decade_data.index.month).quantile(0.75)

    # Plot the percentiles
    fig.add_trace(go.Scatter(x=lower_bound.index, y=lower_bound['Water Temperature'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 25th Percentile', showlegend=True))
    fig.add_trace(go.Scatter(x=upper_bound.index, y=upper_bound['Water Temperature'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 75th Percentile', fill='tonexty', fillcolor=color, showlegend=True))

# Plot 2023 separately
data_2023_water_temp = df_water_temp[df_water_temp.index.year == 2023].sort_index()
if not data_2023_water_temp.empty:
    fig.add_trace(go.Scatter(x=data_2023_water_temp.index.month, y=data_2023_water_temp['Water Temperature'], mode='lines', line=dict(color='#ff954e', width=2), name='Year 2023 Water Temp'))

# Update layout with legend
fig.update_layout(
    title='',
    xaxis=dict(title='Month', tickmode='array', tickvals=months, showgrid=False, ticktext=month_names, title_font=dict(size=16), tickfont=dict(size=16)),
    yaxis=dict(title='Average Water Temperature (°C)', showgrid=False, title_font=dict(size=16), tickfont=dict(size=16)),
    plot_bgcolor='rgba(255,255,255,1)',  # White background
    paper_bgcolor='rgba(255,255,255,1)', 
    legend=dict(
        title='Legend',
        orientation='v',
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top',
        title_font=dict(size=16)
    )
)

fig.show()

fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/water_temp_plot.svg', width = 1250, height = 700)

## Export just shape of the data for visual

In [14]:
fig.update_layout(showlegend=False, xaxis_visible=False, yaxis_visible=False, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', title='')
fig.show()
fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/water_temp_shape.svg', width = 1250, height = 700)

## Final overlayed plot with both parameters

In [17]:
import xarray as xr
import plotly.graph_objects as go
import os
import pandas as pd

# Base directory where the NetCDF files are stored
data_dir = '/Users/casey/Desktop/blue-dot/data/raw'

# Months (1 through 12) and their corresponding names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = list(range(1, 13))

# Decade ranges, starting from 1981
decades = [(1981, 1990), (1991, 2000), (2001, 2010), (2011, 2022)]

# Latitude and longitude bounds for Northern Alaska
lat_bounds = [64, 71]
lon_bounds = [-168, -141]

# Initialize DataFrames for sea ice extent and water temperature
df_sea_ice = pd.DataFrame()
df_water_temp = pd.DataFrame()

# Process each file in the directory for sea ice and water temperature
for file_name in os.listdir(data_dir):
    if file_name.endswith('.nc4'):
        # Extract year and month from file name
        parts = file_name.split('.')
        date_part = parts[-2]  # Format 'YYYYMM'
        year = int(date_part[:4])
        month = int(date_part[4:6])

        file_path = os.path.join(data_dir, file_name)
        ds = xr.open_dataset(file_path)

        # Extract 'FRSEAICE' and 'Var_TSKINWTR' for Northern Alaska region
        sea_ice = ds['FRSEAICE'].sel(lat=slice(*lat_bounds), lon=slice(*lon_bounds))
        water_temp = ds['Var_TSKINWTR'].sel(lat=slice(*lat_bounds), lon=slice(*lon_bounds))

        # Store data in respective DataFrames
        df_sea_ice.at[pd.to_datetime(f'{year}-{month}-01'), 'Sea Ice Extent'] = sea_ice.mean().values
        df_water_temp.at[pd.to_datetime(f'{year}-{month}-01'), 'Water Temperature'] = water_temp.mean().values
        ds.close()

# Create a figure for plotting
fig = go.Figure()

# Custom line and fill colors for each decade
sea_ice_colors = ['#b1ffff', '#00ebeb', '#00c4c4', '#009d9d']
water_temp_colors = {
    (1981, 1990): '#ff9dff',
    (1991, 2000): '#ffd0b1',
    (2001, 2010): '#ffc4ff',
    (2011, 2022): '#ffb889'
}

# Plot for each decade - Sea Ice Extent
for (start_year, end_year), color in zip(decades, sea_ice_colors):
    # Select the data for the decade
    decade_data = df_sea_ice[(df_sea_ice.index.year >= start_year) & (df_sea_ice.index.year <= end_year)]
    
    # Calculate the 25th and 75th percentiles for each month
    lower_bound = decade_data.groupby(decade_data.index.month).quantile(0.25)
    upper_bound = decade_data.groupby(decade_data.index.month).quantile(0.75)

    # Plot the percentiles
    fig.add_trace(go.Scatter(x=lower_bound.index, y=lower_bound['Sea Ice Extent'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 25th Percentile Sea Ice', showlegend=False))
    fig.add_trace(go.Scatter(x=upper_bound.index, y=upper_bound['Sea Ice Extent'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 75th Percentile Sea Ice', fill='tonexty', fillcolor=color, showlegend=False))

# Plot for each decade - Water Temperature
for (start_year, end_year), color in water_temp_colors.items():
    # Select the data for the decade
    decade_data = df_water_temp[(df_water_temp.index.year >= start_year) & (df_water_temp.index.year <= end_year)]
    
    # Calculate the 25th and 75th percentiles for each month
    lower_bound = decade_data.groupby(decade_data.index.month).quantile(0.25)
    upper_bound = decade_data.groupby(decade_data.index.month).quantile(0.75)

    # Plot the percentiles
    fig.add_trace(go.Scatter(x=lower_bound.index, y=lower_bound['Water Temperature'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 25th Percentile Water Temp', yaxis='y2', showlegend=False))
    fig.add_trace(go.Scatter(x=upper_bound.index, y=upper_bound['Water Temperature'], mode='lines', line=dict(color=color, width=1), name=f'{start_year}-{end_year} 75th Percentile Water Temp', fill='tonexty', fillcolor=color, yaxis='y2', showlegend=False))

# Plot the year 2023 for both data types
data_2023_sea_ice = df_sea_ice[df_sea_ice.index.year == 2023].sort_index()
data_2023_water_temp = df_water_temp[df_water_temp.index.year == 2023].sort_index()
if not data_2023_sea_ice.empty:
    fig.add_trace(go.Scatter(x=data_2023_sea_ice.index.month, y=data_2023_sea_ice['Sea Ice Extent'], mode='lines', line=dict(color='#007676', width=2), showlegend=False))
if not data_2023_water_temp.empty:
    fig.add_trace(go.Scatter(x=data_2023_water_temp.index.month, y=data_2023_water_temp['Water Temperature'], mode='lines', line=dict(color='#ff954e', width=2), yaxis='y2', showlegend=False))


# Update layout
fig.update_layout(
    title='',
    xaxis=dict(title='Month', tickmode='array', showgrid=False, tickvals=months, ticktext=month_names, title_font=dict(size=16), tickfont=dict(size=16)),
    yaxis=dict(title='Average Sea Ice Extent', showgrid =False, title_font=dict(size=16), tickfont=dict(size=16)),
    yaxis2=dict(title='Average Water Temperature (°C)', overlaying='y', showgrid=False, side='right', title_font=dict(size=16), tickfont=dict(size=16)),
    plot_bgcolor='rgba(255,255,255,1)',  # White background
    paper_bgcolor='rgba(255,255,255,1)'  # White background
)

fig.show()
fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/final_plot.svg', width = 1250, height = 700)

In [18]:
fig.update_layout(showlegend=False, xaxis_visible=False, yaxis_visible=False, yaxis2_visible = False, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', title='')
fig.show()
fig.write_image('/Users/casey/Desktop/blue-dot/reports/figures/final_shape.svg', width = 1250, height = 700)